In [1]:
import os

import torch
from torch.utils.data import DataLoader

from datasets import load_dataset

from fairscale.nn.model_parallel.initialize import (
    initialize_model_parallel,
    model_parallel_is_initialized,
)
from llama.tokenizer import Tokenizer

from mixture_of_depths.routing_transformer import ModelArgs, MoDTransformer
from mixture_of_depths.train import MoDLlamaTrainer

In [2]:
if not torch.distributed.is_initialized():
    # torch.distributed.init_process_group("nccl")
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12345'
    torch.distributed.init_process_group(backend='gloo', rank=0, world_size=1)
if not model_parallel_is_initialized():
    model_parallel_size = int(os.environ.get("WORLD_SIZE", 1))
    initialize_model_parallel(model_parallel_size)

local_rank = int(os.environ.get("LOCAL_RANK", 0))
torch.cuda.set_device(local_rank)
torch.manual_seed(42)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


c:\Users\Shivaen\anaconda3\envs\MoD\lib\site-packages\torch\__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


## Load Data

In [3]:
dataset = load_dataset("roneneldan/TinyStories", split="validation")

Repo card metadata block was not found. Setting CardData to empty.


In [4]:
dataset

Dataset({
    features: ['text'],
    num_rows: 21990
})

In [5]:
tokenizer = Tokenizer("../llama/tokenizer.model")
def collate_fn(batch):
    bsz = len(batch)
    tokenized_texts = [tokenizer.encode(x['text'], bos=True, eos=True) for x in batch]
    max_text_len = max(len(t) for t in tokenized_texts)

    pad_id = tokenizer.eos_id
    tokens = torch.full((bsz, min(2048, max_text_len)), pad_id, dtype=torch.long)
    for k, t in enumerate(tokenized_texts):
        tokens[k, : len(t)] = torch.tensor(t, dtype=torch.long)[:2048]
    
    return tokens[:,:-1], tokens[:,1:]

dataloader = DataLoader(
    dataset,
    batch_size=4,
    collate_fn=collate_fn,
)

In [6]:
next(iter(dataloader))

(tensor([[    1,  1706,   327,  ...,     2,     2,     2],
         [    1,  9038,  2501,  ..., 29890,  3099, 29889],
         [    1,  9038,  2501,  ...,     2,     2,     2],
         [    1,  9038,  2501,  ...,     2,     2,     2]]),
 tensor([[ 1706,   327, 29889,  ...,     2,     2,     2],
         [ 9038,  2501,   263,  ...,  3099, 29889,     2],
         [ 9038,  2501,   263,  ...,     2,     2,     2],
         [ 9038,  2501,   263,  ...,     2,     2,     2]]))

## Train

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### With MoD and Auxiliary Loss

In [8]:
model_params = ModelArgs(
    dim=512,
    n_layers=6,
    n_heads=8,
    vocab_size=tokenizer.n_words,
    routing=True,
    aux_loss=True
)
model = MoDTransformer(model_params)
count_parameters(model)

53222400

In [12]:
trainer = MoDLlamaTrainer(
    params=model_params,
    model=model,
    tokenizer=tokenizer,
    dataloader=dataloader
)

In [13]:
%%time
trainer.train(
    epochs=5,
    use_aux_loss=model_params.aux_loss,
    use_aux_predictor=model.params.aux_routing
)

Epoch: 1:  18%|█▊        | 1001/5498 [01:23<08:12,  9.13it/s]

Loss at step 1000: 7.058915052413941
Causal Loss at step 1000: 0.08566971867182292


Epoch: 1:  36%|███▋      | 2001/5498 [03:02<05:32, 10.52it/s]

Loss at step 2000: 6.005800765275955
Causal Loss at step 2000: 0.06237742305314169


Epoch: 1:  55%|█████▍    | 3000/5498 [04:40<04:03, 10.27it/s]

...

Loss at step 4000: 2.5672589445412157
Causal Loss at step 4000: 0.010508904797614377


Epoch: 5:  91%|█████████ | 5002/5498 [06:25<00:46, 10.72it/s]

Loss at step 5000: 2.54253048825264
Causal Loss at step 5000: 0.011013002326834248


Epoch: 5: 100%|██████████| 5498/5498 [07:04<00:00, 12.94it/s]


Epoch 5/5 - Loss: 2.5483269950353957
---------- Training Auxiliary Router ----------
CPU times: total: 15min 28s
Wall time: 40min 26s


### With MoD and Auxiliary Router

In [8]:
model_params = ModelArgs(
    dim=512,
    n_layers=6,
    n_heads=8,
    vocab_size=tokenizer.n_words,
    routing=True,
    aux_routing=True
)
model = MoDTransformer(model_params)
count_parameters(model)

53353728

In [9]:
trainer = MoDLlamaTrainer(
    params=model_params,
    model=model,
    tokenizer=tokenizer,
    dataloader=dataloader
)

In [10]:
%%time
trainer.train(
    epochs=5,
    model_dir="./models/MoDLlama_predictor/",
    log_path="./logs/MoDLlama_predictor_log.txt",
    use_aux_loss=model_params.aux_loss,
    use_aux_predictor=model.params.aux_routing
)

Epoch: 1:  18%|█▊        | 1001/5498 [01:15<06:28, 11.59it/s]

Loss at step 1000: 6.880225940465927


Epoch: 1:  36%|███▋      | 2001/5498 [02:30<04:15, 13.70it/s]

Loss at step 2000: 5.789195282936096


Epoch: 1:  55%|█████▍    | 3001/5498 [03:45<03:05, 13.49it/s]

...

Token Predictor Accuracy at step 4000: 0.9683014750480652


Epoch: 1:  91%|█████████ | 5006/5498 [02:15<00:14, 33.50it/s]

Token Predictor Accuracy at step 5000: 0.9649678468704224


Epoch: 1: 100%|██████████| 5498/5498 [02:28<00:00, 36.95it/s]


Epoch 1/1 for aux router - Causal Loss: 0.15164356773336618
CPU times: total: 8min 11s
Wall time: 39min 43s


### Baseline

In [8]:
model_params = ModelArgs(
    dim=512,
    n_layers=6,
    n_heads=8,
    vocab_size=tokenizer.n_words,
    routing=False,
)
model = MoDTransformer(model_params)
count_parameters(model)

53221888

In [9]:
trainer = MoDLlamaTrainer(
    params=model_params,
    model=model,
    tokenizer=tokenizer,
    dataloader=dataloader
)

In [10]:
%%time
trainer.train(
    epochs=5,
    model_dir="./models/BaselineLlama/",
    log_path="./logs/BaselineLlama_log.txt",
)

Epoch: 1:  18%|█▊        | 1001/5498 [01:21<08:10,  9.17it/s]

Loss at step 1000: 6.869257190704346


Epoch: 1:  36%|███▋      | 2001/5498 [02:51<05:13, 11.17it/s]

Loss at step 2000: 5.803991420865059


Epoch: 1:  55%|█████▍    | 3002/5498 [04:21<03:28, 11.97it/s]

...

Loss at step 4000: 2.4853811900913714


Epoch: 5:  91%|█████████ | 5001/5498 [07:23<00:54,  9.06it/s]

Loss at step 5000: 2.4586771923065185


Epoch: 5: 100%|██████████| 5498/5498 [08:08<00:00, 11.25it/s]


Epoch 5/5 - Loss: 2.4640085644052436
CPU times: total: 13min 40s
Wall time: 40min 49s
